# Hekdesh - רשם ההקדשים
This notebook is aimed to 
Data is coming from [רשות התאגידים - רשם ההקדשות](https://data.gov.il/dataset/hekdeshot)  - see [OpenBudget-card](https://github.com/orgs/OpenBudget/projects/3#card-31241405).  

מאגר מידע זה מכיל שלושה קבצים אודות הקדשות ציבוריים: האחד כולל מידע כללי על הקדשות ציבוריים הרשומים פנקס ההקדשות הציבוריים; הקובץ השני כולל מידע אודות יתרת הנכסים של הקדשות ציבוריים; הקובץ השלישי כולל מידע אודות מקרקעין השייכים להקדשות ציבוריים.
קבצי מידע

    רשם ההקדשות - מידע כלליCSV
    תאריך עדכון: 1 מרץ, 2020
    מספר צריכות: 637
    מספר הורדות: 2

    אפשרויות
    רשם ההקדשות - מקרקעין של הקדש ציבוריCSV
    תאריך עדכון: 23 פברואר, 2020
    מספר הורדות: 1

    אפשרויות
    רשם ההקדשות - יתרת הנכסים של הקדש ציבוריCSV
    תאריך עדכון: 23 פברואר, 2020
    מספר צריכות: 379



In [38]:
!pip install dataflows

In [0]:
import pandas as pd
from dataflows import Flow, load, set_type, dump_to_path, printer, join, checkpoint, concatenate, join_with_self

In [0]:
hekdeshGeneral = r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshGeneral.csv"
hekdeshProperty = r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshPropery.csv"
hekdeshFinance =  r"https://raw.githubusercontent.com/sephib/budgetkey-data-pipelines/master/data/hekdesh/hekdeshFinance.csv"  #hekdeshot_nechasimjustice.gov.il.csv"  #r"../data/hekdesh/hekdeshFinance.csv"

## hekdeshGeneral

In [114]:
df = pd.read_csv(hekdeshGeneral,names=["hekdesh_id","record_date","name","purpose","funding","status","col1"]
                 ,dtype ={"hekdesh_id":int,"record_date":object,"name":object,"purpose":object,"funding":'category',"status":'category',"col1":object}
                ,skiprows=1
                ,usecols=["hekdesh_id","record_date","name","purpose","funding","status",] )
df["record_date"] = pd.to_datetime(df["record_date"], format="%d/%m/%Y")
df = df.set_index('hekdesh_id')
df.head(1)

,record_date,name,purpose,funding,status
hekdesh_id,,,,,
590000030,1974-02-21,"אוקס ד~ר דוד וגרדה, קרן על שמם",תמיכה בצעירים וצעירות דתיים המכשירים עצמם לתפק...,הכנסה שוטפת,פעיל


## hekdeshProperty

In [115]:
df_p = pd.read_csv(hekdeshProperty, names=["hekdesh_id","name", "description","gush","chelka","tat_chelka","notes","col1"]
                  ,dtype ={"hekdesh_id":int,"name":object,"description":"category","guhs":int,"chelcka":int,"tat_chelka":int,"notes":object,"col1":object}
                ,usecols = ["hekdesh_id","name", "description","gush","chelka","tat_chelka","notes"],skiprows=1)
df_p.head(1)

,hekdesh_id,name,description,gush,chelka,tat_chelka,notes
0,590000014,יעקבזון דניאל קרן על שמו,מגרש+בית,6188,34,0,יש


In [116]:
# Join hekdeshGeneral with hekdeshProperty
idf_p_cols = ['hekdesh_id','description', 'gush', 'chelka', 'tat_chelka', 'notes']
dfj = df.merge(df_p[idf_p_cols], how='inner',left_index=True,right_on='hekdesh_id')

dfj = dfj.astype({'gush':'Int64', 'chelka':'Int64', 'tat_chelka':'Int64', 'notes':'category'})

dfj.head(1)

,record_date,name,purpose,funding,status,hekdesh_id,description,gush,chelka,tat_chelka,notes
4,1974-02-21,"אוקס ד~ר דוד וגרדה, קרן על שמם",תמיכה בצעירים וצעירות דתיים המכשירים עצמם לתפק...,הכנסה שוטפת,פעיל,590000030,דירה,30007,42,12,יש


## hekdeshFinance

In [117]:
df_f = pd.read_csv(hekdeshFinance
                  ,names=["hekdesh_id","name","currency","value","last_date_update","col3"]
                  ,dtype={"hekdesh_id":int,"name":object,"col1":"category","col2":int,"last_date_update":object,"col3":object}
                   ,skiprows=1
                   ,usecols = ["hekdesh_id","name","currency","value","last_date_update"]
                   ,encoding='utf-8'
                  )
df_f["last_date_update"] = pd.to_datetime(df_f["last_date_update"], format="%d/%m/%Y")
# df_f = df_f.set_index('hekdesh_id')
df_f.head(1)

,hekdesh_id,name,currency,value,last_date_update
0,590000014,יעקבזון דניאל קרן על שמו,ש~ח,210378,2011-12-31


In [118]:
# Each Hekdesh can have several values - even in the same currency - TBD
df_f[df_f['hekdesh_id'].groupby(df_f['hekdesh_id']).transform('size')>1]

,hekdesh_id,name,currency,value,last_date_update
53,590000378,לחובסקי אליעזר ריבקינד יצחק ז~ל הקדש ע~ש,ש~ח,21184,NaT
54,590000378,לחובסקי אליעזר ריבקינד יצחק ז~ל הקדש ע~ש,ש~ח,16221,NaT
60,590000501,סלע יצחק ז~ל קרן מילגות ע~ש,NaN,0,NaT
61,590000501,סלע יצחק ז~ל קרן מילגות ע~ש,ש~ח,0,NaT
63,590000527,רובינשטיין בינימין אבימור משה רובינשטיין קרן מ...,NaN,0,NaT
...,...,...,...,...,...
3430,590038063,קרן ג'ול ואלאונור וואקי,ש~ח,316009,2017-06-30
3431,590038063,קרן ג'ול ואלאונור וואקי,אירו,134495,2017-06-30
3487,590037206,"הק' ע~ש דוד ומריאטה חיימוביצ'י,ז'וזף ורוזלי בז...",דולר ארה~ב,222824,2016-05-12
3488,590037206,"הק' ע~ש דוד ומריאטה חיימוביצ'י,ז'וזף ורוזלי בז...",אירו,37828,2016-05-12


In [100]:
dfpfj = dfj.join(df_f[['currency', 'value', 'last_date_update']])
dfpfj.head(1)

,record_date,name,purpose,funding,status,hekdesh_id,description,gush,chelka,tat_chelka,notes,currency,value,last_date_update
4,1974-02-21,"אוקס ד~ר דוד וגרדה, קרן על שמם",תמיכה בצעירים וצעירות דתיים המכשירים עצמם לתפק...,הכנסה שוטפת,פעיל,590000030,דירה,30007,42,12,יש,ש~ח,1,2012-11-19


# Working with Dataflow

In [0]:

# Map the original column headers (in Hebrew) to the new column names (in English)
COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL = {
    'מספר תיק': 'hekdesh_id',
    'תאריך רישום':'hekdesh_record_date',
    'שם הקדש':'hekdesh_name',
    'מטרות':'hekdesh_purpose',
    'שיטת מימון':'hekdesh_funding',
    'סטטווס תיק':'hekdesh_status',
#     'Unnamed: 6':'col1',
}
COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY = {
    'מספר תיק': 'hekdesh_id_property',
    'שם הקדש':'hekdesh_name_property',
    'תאור הנכס':'hekdesh_description',
    'גוש':'gush',
    'חלקה':'chelcka',
    'תת חלקה':'tat_chelka',
    'הערה במרשם המקרקעין על קיומו של הקדש':'note',
    # 'Unnamed: 7':'col7',
}

COLUMN_HEADERS_MAPPER_HEKDESH_FINANCE = {
    'מספר תיק': 'hekdesh_id_finance',
    'שם הקדש': 'hekdesh_name_finance',
    'תאור מטבע':'currency',
    'סכום':'value', 
    'נכון לתאריך':'last_date_value_update',
}


def clear_bool_values(package):
    for res in package.pkg.descriptor['resources']:
        for field in res['schema']['fields']:
            if 'falseValues' in field:
                del field['falseValues']
            if 'trueValues' in field:
                del field['trueValues']
    yield package.pkg
    yield from package

def _get_columns_mapping_dict(columns_rename):
    """
    Prepares the dict object for the concatenate function, which 'creates' new column names for existing column data
    Returns:
        dict: A dict of format {'new_column_header': ['original_column_header]},
    """
    columns_mapping_dict = {}
    for original_header in columns_rename:
        new_header = columns_rename[original_header]
        columns_mapping_dict[new_header] = [original_header]
    return columns_mapping_dict

In [0]:
# Flow with 2 resources
f=Flow(
        load(hekdeshProperty, cast_strategy=load.CAST_TO_STRINGS),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY)
                   , target=dict(name='hekdesh-property') #, resources='hekdesh-property'  #
        ),
        set_type('hekdesh_id_property', type='integer'),
        load(hekdeshGeneral, cast_strategy=load.CAST_TO_STRINGS, resources='hekdesh-general'),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL)
                     , resources=-1,   target=dict(name='hekdesh-general'),  # , resources='hekdesh-general'
        ),
        set_type('hekdesh_id', type='integer'),
        # load(hekdeshProperty, cast_strategy=load.CAST_TO_STRINGS
        #      , resources='hekdesh-finance'),
        #     concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_FINANCE)
        #                  ,target=dict(name='hekdesh-finance'),  # , resources='hekdesh-general'
        #     ),
        # set_type('hekdesh_id_finance', type='integer'),
        checkpoint('load_data'),
        join(
                'hekdesh-property', ['hekdesh_id_property'],  # Source resource 
                'hekdesh-general', ['hekdesh_id'],   # Target resource
                dict(
                hekdesh_description = {'name' : 'hekdesh_description'},
                gush = {'name' : 'gush'},
                chelcka = {'name': 'chelcka'},
                tat_chelka = {'name': 'tat_chelka'},
                note = {'name': 'note'},
                ),
                mode='half-outer'  #''inner'   # Don't add new fields, remove unmatched rows
            ),
        #  join_with_self('hekdesh-general',
        #                 ['hekdesh_id'],
        #                 fields=( {
        #                     'gush': {
        #                         'name': 'gush',
        #                         'aggregate': 'array'
        #                     },
        #                     '*': {
        #                         'aggregate': 'first'
        #                     },
        #                 }),
#                       dict(hekdesh_description=None, value=dict(aggregate='first')),
#                       dict(gush=None, value=dict(aggregate='array'))  
                        
                      #  ),
#         join(
#             'hekdesh-finance', ['hekdesh_id_finance'],  # Source resource 
#             'hekdesh-general', ['hekdesh_id'],   # Target resource
#             dict(
#                 currency = {'name' : 'currency'},
#                 value = {'name' : 'value'},
#                 last_date_value_update = {'name' : 'last_date_value_update'}
#                 ),
#                 mode='half-outer'  #''inner'   # Don't add new fields, remove unmatched rows
#             ),
#             set_type('hekdesh_record_date', type='date', format='%d/%m/%Y'),
        
            printer(num_rows=1)
)

In [122]:
f.process()[1];

using checkpoint data from .checkpoints/load_data
hekdesh-general:
#       hekdesh_id  hekdesh_record_date    hekdesh_name                                hekdesh_purpose                                                                                           hekdesh_funding    hekdesh_status    chelcka      gush         hekdesh_description    note        tat_chelka
         (integer)  (string)               (string)                                    (string)                                                                                                  (string)           (string)          (integer)    (integer)    (string)               (string)    (integer)
----  ------------  ---------------------  ------------------------------------------  --------------------------------------------------------------------------------------------------------  -----------------  ----------------  -----------  -----------  ---------------------  ----------  ------------
1        590000030  21/02/

In [129]:
# with array
f=Flow(
        load(hekdeshProperty, cast_strategy=load.CAST_TO_STRINGS),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY)
                   , target=dict(name='hekdesh-property') #, resources='hekdesh-property'  #
        ),
        set_type('hekdesh_id_property', type='integer'),
        load(hekdeshGeneral, cast_strategy=load.CAST_TO_STRINGS, resources='hekdesh-general'),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL)
                     , resources=-1,   target=dict(name='hekdesh-general'),  # , resources='hekdesh-general'
        ),
        set_type('hekdesh_id', type='integer'),
        checkpoint('load_data'),
        join(
                'hekdesh-property', ['hekdesh_id_property'],  # Source resource 
                'hekdesh-general', ['hekdesh_id'],   # Target resource
                dict(
                hekdesh_description = {'name' : 'hekdesh_description'},
                gush = {'name' : 'gush'},
                chelcka = {'name': 'chelcka'},
                tat_chelka = {'name': 'tat_chelka'},
                note = {'name': 'note'},
                ),
                mode='half-outer'  #''inner'   # Don't add new fields, remove unmatched rows
            ),
        join_with_self('hekdesh-general',
                        ['hekdesh_id'],
                        fields=( {
                            'gush': {
                                'name': 'gush',
                                'aggregate': 'array'
                            },
                            '*': {
                                'aggregate': 'first'
                            },
                        }),                      
        ),  
        printer(num_rows=1)
)
f.process()[1];

/usr/local/lib/python3.6/dist-packages/dataflows/processors/join.py:180: UserWarning: For the `join` processor the `full=True` flag is deprecated. Please use the "mode" parameter instead.
  UserWarning)


using checkpoint data from .checkpoints/load_data
hekdesh-general:
#     chelcka      gush       hekdesh_description    hekdesh_funding      hekdesh_id  hekdesh_name                    hekdesh_purpose                                                                                           hekdesh_record_date    hekdesh_status    note        tat_chelka
      (integer)    (array)    (string)               (string)              (integer)  (string)                        (string)                                                                                                  (string)               (string)          (string)    (integer)
----  -----------  ---------  ---------------------  -----------------  ------------  ------------------------------  --------------------------------------------------------------------------------------------------------  ---------------------  ----------------  ----------  ------------
1     None         []         None                   הכנסה שוטפת    

# How to add 3 resources?

In [0]:
# H

In [0]:
# Workspace

In [0]:
d = {
    'מספר תיק': 'hekdesh_id_finance',
    'שם הקדש': 'hekdesh_name_finance',
    'תאור מטבע':'currency',
    'סכום':'value', 
    'נכון לתאריך':'last_date_value_update',
}
[f'{i} = {{\'name\' : \'{i}\'}}' for i in d.values()]

In [125]:
Flow(
        load(r'data/hekdesh/hekdeshProperty.csv', cast_strategy=load.CAST_TO_STRINGS),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_PROPERTY)
                   , target=dict(name='hekdesh-property')# , resources='hekdesh-property'  #
        ),
        # set_type('hekdesh_id_property', type='integer'),
        # set_type('hekdesh_name_property', type='string'),
        # set_type('hekdesh_description', type='string'),
        # set_type('gush', type='integer'),
        # set_type('chelcka', type='integer'),
        # set_type('tat_chelka', type='integer'),
        # set_type('note', type='string'),
        # update_resource(**{'dpp:streaming': True}, resources='hekdesh-property'),
        # set_primary_key(['hekdesh_id_property'], resources='hekdesh-property'),

        load(r'data/hekdesh/hekdeshGeneral.csv', cast_strategy=load.CAST_TO_STRINGS, resources='hekdesh-general'),
        concatenate(_get_columns_mapping_dict(COLUMN_HEADERS_MAPPER_HEKDESH_GENERAL)
                     , resources=-1,   target=dict(name='hekdesh-general'),  # , resources='hekdesh-general'
        ),
        # # set_type('hekdesh_id', type='integer'),
        # # set_type('hekdesh_record_date', type='date', format='%d/%m/%Y'),
        # # set_type('hekdesh_name', type='string'),
        # # set_type('hekdesh_purpose', type='string'),
        # # set_type('hekdesh_funding', type='string'),
        # # set_type('hekdesh_status', type='string'),
        # # update_resource(**{'dpp:streaming': True}, resources='hekdesh-general'),
        # set_primary_key(['hekdesh_id'], resources='hekdesh-general'),
        
        join(
            'hekdesh-property', ['hekdesh_id_property'],  # Source resource 
            'hekdesh-general', ['hekdesh_id'],   # Target resource
            # field={*},
            mode='inner'   # Don't add new fields, remove unmatched rows
        ),
        printer(num_rows=3),
        # printer(num_rows=3, tablefmt='html'),
    )

In [0]:

def set_schema_hekdeshGeneral(package):
    # set schema for fields from csv file
    package.pkg.descriptor['resources'][0]['schema']['fields'].append(
        {"hekdesh_id":int,"record_date":object,"name":object,"purpose":object,"funding":'category',"status":'category',"col1":object}
    )
    # Must yield the modified datapackage
    yield package.pkg
    # And its resources
    yield from package

def find_double_names(package):
    # Remove the emmies resource -  we're going to consume it now
    package.pkg.remove_resource('hekdesh_general')
    # Must yield the modified datapackage
    yield package.pkg
    # Now iterate on all resources
    resources = iter(package)
    # Emmies is the first - read all its data and create a set of winner names
    general = next(resources)
    general_winners = set(
        map(lambda x: x['מספר תיק','תאריך רישום'],
            filter(lambda x: x['winner'],
                   general))
    )
    # Oscars are next - filter rows based on the emmy winner set
    academy = next(resources)
    yield filter(lambda row: (row['Winner'] and
                              row['Name'] in emmy_winners),
                 academy)
    # important to deque generators to ensure finalization steps of previous processors are executed
    yield from resources

Flow(
    # Same one as above
    load(hekdeshGeneral, name='hekdesh_general'
#          , override_schema={"hekdesh_id":int,"record_date":object,"name":object,"purpose":object,"funding":'category',"status":'category',"col1":object}
         
        ),
    load(hekdeshProperty, name='hekdesh_property'),
#     checkpoint('hekdesh_general_property'),
#     find_double_names,
#     set_schema_hekdeshGeneral,
#     add_is_guitarist_column,
    dump_to_path('hekdesh_general_property'),
    join(
        'hekdesh_general', ['מספר תיק'],  # Source resource
        'hekdesh_property', ['מספר תיק'], # Target resource
        mode='full-outer'   # Don't add new fields, remove unmatched rows
    ),
    printer(num_rows=1, tablefmt='html')
).process()[1]